# OpenStreetMap Data Case Study
## Map Area
Los-Angeles, California.<br/>
https://mapzen.com/data/metro-extracts/metro/los-angeles_california/ <br/>
This city is one of my favorate cities in US, so I’m more interested to see what database querying reveals.

## Problems Encountered in the Map
Here are x main problems that I noticed after looking at a small sample size of the data.
- "Type:key" patterned "k" value in tags
- Abbreviated street type
- Post code inconsistency problem
- Duplication problem of node_id field
- Region included much more than los-angeles itself 
<br />
<br />
Next, I am going to approach those problems step by step and the prodedures are shown with code below.

### 'k' Value problem of tags
First, let us dive into k values of the tags.

In [16]:
'''First of all, the functions of data operating is imported.'''
from tools import *

In [2]:
from dbi import *
PATH_HEADER="F:\Hsiao's studying\Computing\Udacity P3\\"
dbi=dbInterface(PATH_HEADER+'mapdata.db')

In [3]:
'''Next, the k values with problem are reported below.'''
probKey=find_problem_key(OSM_PATH)
for i in probKey:
    print i,':',len(probKey[i])

problemchars : 0
lower : 35956
other : 704
lower_colon : 30367


The result above shows several facts:
- No problem chars appears in sample dataset(like ?#*)
- About half of the k values are in all-lower case
- Another half are in lower_colon(which looks like "type:key")

Solution: Before gathering the osm data into csv and db, seperate the 'k' field into fields 'type' and 'key'.<br/>
For those regular k values, just assign the type as 'regular' and key as the value provided;<br/>
For lower_colon cases, assign the type field with the content before the ':' sign which is first represented in the value and the key field with the content after the ':' sign.

In [11]:
def split_k_value(rawAttrib):
    '''this is the solution.
       'k,v' fields are seperated in 'type,key,value' fields.
    '''
    tagAttrib={}
    tagAttrib['value']=rawAttrib['v']
    kContent=rawAttrib['k'].split(':')
    if len(kContent)==1:
        kContent=['regular']+kContent # add the regular
    tagAttrib['type']=kContent[0]
    tagAttrib['key']=':'.join(kContent[1:])
    return tagAttrib

### Abbreviated Street Name Problem
Next let us dive into the problem of abbreviated street types which is the last word of the street name.
Some of the name are type-abbreviated for example 'St' is written instead of 'Street'.

In [8]:
tc=type_count(OSM_PATH) # call the function which count the types of the streets
tc=sorted(tc.items(),key=lambda i:i[0]) # to make them output in alphabetic order
for i in tc:
    print i[0],':',i[1]

Aguacate : set(['Via del Aguacate'])
Alamitos : set(['Alamitos'])
Albercon : set(['Avenida Albercon'])
Alevera : set(['Alevera'])
Alondra : set(['Alondra'])
Amigos : set(['Los Amigos'])
Asbury : set(['Asbury'])
Ashfield : set(['Ashfield'])
Atascadero : set(['Atascadero'])
Autumn : set(['Autumn'])
Av : set(['1341 Euclid St @Trask Av'])
Ave : set(['Newhal Ave', 'Quince Ave', 'Bison Ave', 'Sandalwood Ave'])
Aven : set(['Chapman Aven'])
Avondale : set(['Avondale'])
Barbara : set(['Avenida Santa Barbara'])
Bend : set(['Upper Bend'])
Birmingham : set(['Birmingham'])
Blvd : set(['Wilshire Blvd', 'Foothill Blvd', 'West Coast Blvd'])
Boda : set(['Boda'])
Bolas : set(['Calle Las Bolas'])
Borrego : set(['Borrego'])
Bragg : set(['Bragg'])
Brillantez : set(['Brillantez'])
Broadway : set(['South Broadway'])
Brook : set(['Kari Brook'])
Bumblebee : set(['Bumblebee'])
Cabrillo : set(['Calle Cabrillo'])
Calais : set(['Calais'])
Cambridge : set(['Cambridge'])
Canada : set(['West Canada'])
Carson : set(['

Plenty of non-English words represented in the results above and this is very California.<br/>
Some of them looks like abbreviation(like Pl, Ln and Mar) but I am not that sure so I decide to ignore them.
My name updating process will focus on streets, roads, avenues and circles.<br/><br/>
To solve this problem, if the k value of tag is 'addr:street', its value would be changed using the function and mapping below.

In [10]:
# updating dictionary
mapping = { "St": "Street",
            "St.": "Street",
           'Av':"Avenue",
            "Ave":"Avenue",
            'Ave.':'Avenue',
           'Aven':'Avenue',
            'Rd':'Road',
            'Rd.':'Road',
           'Cir':'Circle'
            }

In [12]:
def update_name(name, mapping):
    '''change the last word of the street's name from abbr to fullname'''
    name=name.split()
    if name[-1] in mapping:
        name[-1]=mapping[name[-1]]
    return ' '.join(name)

### Postcode Inconsistency Problem
Postal code strings posed a different sort of problem, forcing a decision to strip all leading and trailing characters before and after the main 5­digit zip code. This effectively dropped all leading state characters (such as “AAXXXXX”) and 4­digit zip code extensions following a hyphen (“XXXXX-XXXX”). This 5­digit restriction allows for more consistent queries.<br/><br/>
To solve this, if the k value of the tag is 'addr:postcode', the value of it would be changed using the function below.

In [18]:
def update_postcode(postcode):
    # change the postcode to 'XXXXX' in pattern 'XXXXX-XXXX' or 'AAXXXXX' (A means alpha and X means digit)
    postcode=str(postcode)
    if len(postcode)==5:
        return postcode
    elif len(postcode)!=5 and postcode[0].isdigit():
        return postcode[:5]
    elif not postcode[0].isdigit():
        for i in range(len(postcode)):
            if postcode[i].isdigit():
                return postcode[i:i+5]
    

### NodeID Duplication Problem
In the process of importing the csv data into database, I found that with the primary key restriction of node id, most nodes are error-raised(sqlite3.IntegrityError). This is caused by duplication of the node-id inside the csv file. I found that all the updating of a node is recorded in the original xml file and only the latest update is required to be imported to database.<br/>
To solve this problem, the code below with 'try-catch' statement is presented.
Because the later updating always appear priorier than the pre ones, so just skip those pre updates which cause the integrity error.

In [ ]:
try:
    to_db=map(self.unicode_change,i.values()) # map the function which change the bytestring to unicode string
    self.cu.execute("INSERT INTO %s(%s) VALUES (%s);"%(table_name,','.join(i.keys()),','.join(['?']*len(to_db))),to_db)
except Exception as e:
    if type(e)==sqlite3.IntegrityError: # pass the integrity errors
        pass
    else:
        print type(e),':',e # report other errors
        print i.values()
        time.sleep(0.3)

### Sort Cities By Count
These results confirmed my suspicion that this metro extract would perhaps be more aptly named “Metrolina” or the “Los-Angeles Metropolitan Area” for its inclusion of surrounding cities in the sprawl.

In [3]:
# sort cities
dbi.select_and_repr(
'''
SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) tags
WHERE tags.key LIKE '%city'
GROUP BY tags.value
ORDER BY count DESC
LIMIT 0,15;
'''
) 

(u'Irvine', 25004)
(u'Lake Forest', 14844)
(u'Fallbrook', 12515)
(u'Norco', 7284)
(u'San Clemente', 4113)
(u'Los Angeles', 1997)
(u'Tustin', 1947)
(u'Trabuco Canyon', 1659)
(u'Foothill Ranch', 1641)
(u'Pomona', 931)
(u'Corona', 800)
(u'Rancho Cucamonga', 672)
(u'Pauma Valley', 610)
(u'Laguna Niguel', 582)
(u'Ontario', 551)


## Data Overview
This section contains basic statistics about the dataset, and the SQL queries used to gather them.
### File sizes

In [ ]:
file sizes:
    los-angeles_california.osm ......... 4.07 GB
    mapdata.db ......................... 2.43 GB
    nodes.csv .......................... 1.62 GB
    nodes_tags.csv ..................... 33.8 MB
    ways.csv ............................ 124 MB
    ways_tags.csv ....................... 513 MB
    ways_nodes.cv ....................... 414 MB  

### Number of nodes

In [4]:
# number of nodes
dbi.select_and_repr('SELECT COUNT(*) FROM nodes;')

(18910668,)


### Number of ways

In [5]:
# number of ways
dbi.select_and_repr('SELECT COUNT(*) FROM ways;')

(1947075,)


### Number of unique users

In [3]:
# number of unique users
dbi.select_and_repr('SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;')

(3387,)


### Top 10 contributing user

In [4]:
# top 10 contributing user
dbi.select_and_repr('''SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10;''')

(u'manings_labuildings', 2112352)
(u'schleuss_imports', 1559776)
(u'Jothirnadh_labuildings', 1316835)
(u'calfarome_labuilding', 693222)
(u'nammala_labuildings', 604213)
(u'Luis36995_labuildings', 596527)
(u'saikabhi_LA_imports', 570205)
(u'The Temecula Mapper', 548130)
(u'planemad_imports', 529421)
(u'woodpeck_fixbot', 510267)


### Number of users that appear only once

In [5]:
# num of users that only appear once
dbi.select_and_repr(''' SELECT COUNT(*) 
FROM
    (SELECT e.user, COUNT(*) as num
     FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
     GROUP BY e.user
     HAVING num=1)  u;''')

(735,)


## Additional Ideas
### Contributor statistics
The contribution of users is skewed but not that skewed, and most of the data is provided with the users which has '_labuildings' inside their usernames. Here are some user percentage statistics:
- Top user contribution percentage('manings_labuildings'): 10.13%
- Combined top 10 users contribution: 43.35%
- Combined user with '_labuildings' contribution: 33.03%
It is obvious that 'labuildings' usergroup contributes about one third of the map data.

## Additional Data Exploration
### Top 10 appearing amentities

In [7]:
# top 10 appearing amenities
dbi.select_and_repr('''SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 10;''')

(u'place_of_worship', 3712)
(u'school', 2926)
(u'restaurant', 2528)
(u'fast_food', 1405)
(u'fuel', 814)
(u'cafe', 734)
(u'fountain', 686)
(u'bench', 633)
(u'toilets', 578)
(u'parking', 510)


### Top 3 religions

In [10]:
# top 3 religions
dbi.select_and_repr(
''' SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 3;''')

(u'christian', 3466)
(u'jewish', 35)
(u'buddhist', 19)


### Top 10 most popular cuisines

In [9]:
# top 10 most popular cuisines
dbi.select_and_repr('''SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;''')

(u'mexican', 200)
(u'american', 172)
(u'pizza', 137)
(u'italian', 89)
(u'chinese', 87)
(u'sushi', 66)
(u'japanese', 64)
(u'thai', 64)
(u'burger', 58)
(u'sandwich', 45)


## Conclusion
After the review of this data, a summary is made with several points:
- Plenty of abbreviations and non-English words are represented in street names
- All the updates are included in file but only the latest one is used in following statistics
- Most of the surrounding cities in the sprawl are included in this map data
- The contribution of user is kind of skewed with top 10 contribute nearly half of the data and 'labuildings' usergroup contributes one third of the data